In [3]:
!pip -q install fastapi==0.111.0 uvicorn==0.30.1 nest_asyncio==1.6.0 \
               pandas==2.2.2 numpy==1.26.4 pydantic==2.8.2 requests==2.32.3 httpx==0.27.0


In [63]:
!pip uninstall -y uvloop httptools nest_asyncio


Found existing installation: nest-asyncio 1.6.0
Uninstalling nest-asyncio-1.6.0:
  Successfully uninstalled nest-asyncio-1.6.0


In [1]:
!pip install cloudflared


In [2]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


(Reading database ... 126375 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.8.0) over (2025.8.0) ...
Setting up cloudflared (2025.8.0) ...
Processing triggers for man-db (2.10.2-1) ...


In [29]:
!pip install pydantic-settings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.9 MB/s eta 0:00:00


In [1]:
!pip install --force-reinstall numpy==1.26.4 pandas==2.2.2


  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstal

In [26]:
!pip install pymongo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.0 MB/s eta 0:00:00


In [32]:
!pip install pydantic-settings


In [48]:
import os, textwrap

BASE = "/content/ai_scoring_server"
os.makedirs(f"{BASE}/app/models", exist_ok=True)
os.makedirs(f"{BASE}/app/services", exist_ok=True)
os.makedirs(f"{BASE}/app/utils", exist_ok=True)

main_py = r'''
import time
from datetime import datetime
from typing import Any, Dict

from fastapi import FastAPI, Body
from fastapi.responses import JSONResponse

from .models.dex_model import process_wallet_complete, preprocess_dex_transactions
from .services.kafka_service import run_kafka_loop # Import kafka_service

app = FastAPI(title="AI Scoring Server", version="1.0.0")

START_TIME = time.time()
STATS = {"processed": 0, "success": 0, "failure": 0, "last_ms": 0.0}

@app.get("/")
def root() -> Dict[str, Any]:
    return {"service": "ai-scoring-server", "version": "1.0.0", "status": "ok"}

@app.get("/api/v1/health")
def health() -> Dict[str, Any]:
    uptime = time.time() - START_TIME
    return {"status": "healthy", "uptime_seconds": round(uptime, 2)}

@app.get("/api/v1/stats")
def stats() -> Dict[str, Any]:
    uptime = time.time() - START_TIME
    return {
        "processed": STATS["processed"],
        "success": STATS["success"],
        "failure": STATS["failure"],
        "last_ms": STATS["last_ms"],
        "uptime_seconds": round(uptime, 2),
    }

def _to_native(obj):
    """Recursively convert numpy/pandas types to native Python types for JSON safety."""
    if isinstance(obj, dict):
        return {k: _to_native(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [_to_native(v) for v in obj]
    t = str(type(obj)).lower()
    if "numpy" in t:
        try: return obj.item()
        except Exception: return float(obj)
    if hasattr(obj, "to_dict") and "pandas" in t:
        return _to_native(obj.to_dict())
    return obj

@app.post("/api/v1/score")
def score_wallet(payload: Dict[str, Any] = Body(...)) -> JSONResponse:
    t0 = time.time()
    STATS["processed"] += 1
    try:
        final_score, features = process_wallet_complete(payload)
        # Model signals empty / invalid
        if isinstance(features, dict) and features.get("error"):
            STATS["failure"] += 1
            processing_time_ms = int((time.time() - t0) * 1000)
            STATS["last_ms"] = processing_time_ms
            failure_msg = {
                "wallet_address": payload.get("wallet_address", ""),
                "error": features["error"],
                "timestamp": int(datetime.now().timestamp()),
                "processing_time_ms": processing_time_ms,
                "categories": [{
                    "category": "dexes",
                    "error": features["error"],
                    "transaction_count": 0
                }]
            }
            return JSONResponse(content=failure_msg, status_code=200)

        df = preprocess_dex_transactions(payload)
        processing_time_ms = int((time.time() - t0) * 1000)
        STATS["last_ms"] = processing_time_ms
        STATS["success"] += 1

        success_msg = {
            "wallet_address": payload.get("wallet_address", ""),
            "zscore": f"{final_score:.18f}",
            "timestamp": int(datetime.now().timestamp()),
            "processing_time_ms": processing_time_ms,
            "categories": [{
                "category": "dexes",
                "score": final_score,
                "transaction_count": int(len(df)),
                "features": {k: v for k, v in features.items() if k not in ["score_breakdown", "user_tags"]}
            }]
        }
        return JSONResponse(content=_to_native(success_msg), status_code=200)

    except Exception as e:
        STATS["failure"] += 1
        processing_time_ms = int((time.time() - t0) * 1000)
        STATS["last_ms"] = processing_time_ms
        failure_msg = {
            "wallet_address": payload.get("wallet_address", ""),
            "error": f"Unhandled error: {e}",
            "timestamp": int(datetime.now().timestamp()),
            "processing_time_ms": processing_time_ms,
            "categories": [{
                "category": "dexes",
                "error": "Unhandled error",
                "transaction_count": 0
            }]
        }
        return JSONResponse(content=failure_msg, status_code=500)
'''

dex_model_py = r'''
from __future__ import annotations
from typing import Dict, Any, Tuple, List
from datetime import datetime
import pandas as pd
import numpy as np

def preprocess_dex_transactions(wallet_data: Dict) -> pd.DataFrame:
    rows: List[Dict[str, Any]] = []
    wallet_address = wallet_data.get("wallet_address", "")
    for category_data in wallet_data.get("data", []):
        if category_data.get("protocolType") != "dexes":
            continue
        for tx in category_data.get("transactions", []):
            row = {
                "wallet_address": wallet_address,
                "document_id": tx.get("document_id", ""),
                "action": tx.get("action", ""),
                "timestamp": tx.get("timestamp", 0),
                "protocol": tx.get("protocol", ""),
                "pool_id": tx.get("poolId", ""),
                "pool_name": tx.get("poolName", ""),
            }
            action = row["action"]
            if action == "swap":
                token_in = tx.get("tokenIn", {}) or {}
                token_out = tx.get("tokenOut", {}) or {}
                row["amount_usd"] = float(max(token_in.get("amountUSD", 0.0), token_out.get("amountUSD", 0.0)))
                row["token_in_symbol"] = token_in.get("symbol", "")
                row["token_out_symbol"] = token_out.get("symbol", "")
            elif action in ("deposit", "withdraw"):
                token0 = tx.get("token0", {}) or {}
                token1 = tx.get("token1", {}) or {}
                # Match challenge expectations exactly:
                # deposit USD = token0.amountUSD (fallback to token1, else sum)
                # withdraw USD = token0.amountUSD + token1.amountUSD
                if action == "deposit":
                    if "amountUSD" in token0:
                        amt = float(token0.get("amountUSD", 0.0))
                    elif "amountUSD" in token1:
                        amt = float(token1.get("amountUSD", 0.0))
                    else:
                        amt = float(token0.get("amountUSD", 0.0)) + float(token1.get("amountUSD", 0.0))
                else:
                    amt = float(token0.get("amountUSD", 0.0)) + float(token1.get("amountUSD", 0.0))
                row["amount_usd"] = amt
                row["token0_symbol"] = token0.get("symbol", "")
                row["token1_symbol"] = token1.get("symbol", "")
            rows.append(row)
    df = pd.DataFrame(rows)
    if not df.empty:
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="s", errors="coerce")
    return df

def calculate_holding_time(deposits: pd.DataFrame, withdraws: pd.DataFrame) -> float:
    if deposits.empty:
        return 0.0
    holding_times = []
    now = datetime.now().timestamp()
    for _, dep in deposits.iterrows():
        t_dep = float(dep["timestamp"])
        later_withdraws = withdraws[withdraws["timestamp"] > t_dep]
        if not later_withdraws.empty:
            t_wd = float(later_withdraws["timestamp"].min())
            holding_times.append((t_wd - t_dep) / 86400.0)
        else:
            holding_times.append((now - t_dep) / 86400.0)
    return float(np.mean(holding_times)) if holding_times else 0.0

def calculate_lp_features(df: pd.DataFrame) -> Dict[str, float]:
    if df.empty:
        return {}
    deposits = df[df["action"] == "deposit"]
    withdraws = df[df["action"] == "withdraw"]
    total_deposit_usd = float(deposits["amount_usd"].sum()) if not deposits.empty else 0.0
    total_withdraw_usd = float(withdraws["amount_usd"].sum()) if not withdraws.empty else 0.0
    return {
        "total_deposit_usd": total_deposit_usd,
        "total_withdraw_usd": total_withdraw_usd,
        "num_deposits": int(len(deposits)),
        "num_withdraws": int(len(withdraws)),
        "withdraw_ratio": (total_withdraw_usd / total_deposit_usd) if total_deposit_usd > 0 else 0.0,
        "avg_hold_time_days": calculate_holding_time(deposits, withdraws),
        "account_age_days": ((df["timestamp"].max() - df["timestamp"].min()) / 86400.0) if len(df) else 0.0,
        "unique_pools": int(df["pool_id"].nunique() if "pool_id" in df else 0),
    }

def calculate_token_diversity(swaps: pd.DataFrame) -> int:
    if swaps.empty:
        return 0
    stable = {"USDC", "USDT", "DAI", "LUSD", "USDP", "TUSD", "FRAX"}
    tokens = set(swaps.get("token_in_symbol", pd.Series(dtype=str)).dropna()) | set(swaps.get("token_out_symbol", pd.Series(dtype=str)).dropna())
    stable_count = len(tokens & stable)
    volatile_count = len(tokens) - stable_count
    return int(min(stable_count * 10 + volatile_count * 15, 150))

def calculate_swap_frequency(swaps: pd.DataFrame) -> float:
    if swaps.empty or len(swaps) < 2:
        return 0.0
    s = swaps.sort_values("timestamp")
    diffs = s["timestamp"].diff().dropna() / 3600.0
    avg_h = float(diffs.mean())
    if avg_h <= 1: return 100.0
    if avg_h <= 24: return 80.0
    if avg_h <= 168: return 60.0
    if avg_h <= 720: return 40.0
    return 20.0

def calculate_swap_features(df: pd.DataFrame) -> Dict[str, float]:
    if df.empty:
        return {}
    swaps = df[df["action"] == "swap"]
    if swaps.empty:
        return {
            "total_swap_volume": 0.0, "num_swaps": 0, "unique_pools_swapped": 0,
            "avg_swap_size": 0.0, "token_diversity_score": 0, "swap_frequency_score": 0.0,
        }
    total_swap_volume = float(swaps["amount_usd"].sum())
    num_swaps = int(len(swaps))
    unique_pools_swapped = int(swaps["pool_id"].nunique())
    avg_swap_size = float(total_swap_volume / num_swaps) if num_swaps else 0.0
    token_diversity_score = calculate_token_diversity(swaps)
    swap_frequency_score = calculate_swap_frequency(swaps)
    return {
        "total_swap_volume": total_swap_volume,
        "num_swaps": num_swaps,
        "unique_pools_swapped": unique_pools_swapped,
        "avg_swap_size": avg_swap_size,
        "token_diversity_score": token_diversity_score,
        "swap_frequency_score": swap_frequency_score,
    }

def calculate_lp_score(features: Dict[str, float]) -> tuple[float, Dict[str, Any]]:
    if not features:
        return 0.0, {}
    volume_score = min(features["total_deposit_usd"] / 10000 * 300, 300)
    frequency_score = min(features["num_deposits"] * 20, 200)
    retention_score = max(0.0, (1 - features["withdraw_ratio"]) * 250)
    holding_score = min(features["avg_hold_time_days"] / 30 * 150, 150)
    diversity_score = min(features["unique_pools"] * 20, 100)
    total = float(volume_score + frequency_score + retention_score + holding_score + diversity_score)
    return total, {
        "volume_score": volume_score,
        "frequency_score": frequency_score,
        "retention_score": retention_score,
        "holding_score": holding_score,
        "diversity_score": diversity_score,
        "total_lp_score": total,
    }

def calculate_swap_score(features: Dict[str, float]) -> tuple[float, Dict[str, Any]]:
    if not features:
        return 0.0, {}
    volume_score = min(features["total_swap_volume"] / 50000 * 250, 250)
    frequency_score = min(features["num_swaps"] * 10, 200)
    diversity_score = features["token_diversity_score"]
    activity_score = features["swap_frequency_score"]
    pool_diversity_score = min(features["unique_pools_swapped"] * 25, 100)
    total = float(volume_score + frequency_score + diversity_score + activity_score + pool_diversity_score)
    return total, {
        "volume_score": volume_score,
        "frequency_score": frequency_score,
        "diversity_score": diversity_score,
        "activity_score": activity_score,
        "pool_diversity_score": pool_diversity_score,
        "total_swap_score": total,
    }

def generate_user_tags(lp_features: Dict[str, Any], swap_features: Dict[str, Any]) -> List[str]:
    tags: List[str] = []
    td = lp_features.get("total_deposit_usd", 0)
    if td > 100000: tags.append("Whale LP")
    elif td > 10000: tags.append("Large LP")
    elif td > 1000: tags.append("Medium LP")
    elif td > 0: tags.append("Small LP")
    ht = lp_features.get("avg_hold_time_days", 0)
    if ht > 90: tags.append("Long-term Holder")
    elif ht > 30: tags.append("Medium-term Holder")
    elif ht > 0: tags.append("Short-term Holder")
    sv = swap_features.get("total_swap_volume", 0)
    if sv > 500000: tags.append("Whale Trader")
    elif sv > 50000: tags.append("Large Trader")
    elif sv > 5000: tags.append("Active Trader")
    elif sv > 0: tags.append("Casual Trader")
    ns = swap_features.get("num_swaps", 0)
    if ns > 100: tags.append("High Frequency Trader")
    elif ns > 20: tags.append("Regular Trader")
    if swap_features.get("token_diversity_score", 0) > 100:
        tags.append("Diversified Trader")
    elif lp_features.get("unique_pools", 0) > 3:
        tags.append("Multi-Pool LP")
    return tags

def calculate_final_score(lp_score: float, swap_score: float, lp_features: Dict[str, Any], swap_features: Dict[str, Any]) -> tuple[float, Dict[str, Any]]:
    final = float(lp_score * 0.6 + swap_score * 0.4)
    tags = generate_user_tags(lp_features, swap_features)
    features = {**lp_features, **swap_features, "user_tags": tags, "lp_score": lp_score, "swap_score": swap_score, "final_score": final}
    return final, features

def process_wallet_complete(wallet_data: Dict[str, Any]) -> Tuple[float, Dict[str, Any]]:
    df = preprocess_dex_transactions(wallet_data)
    if df.empty:
        return 0.0, {"error": "No valid transactions found"}
    lp_features = calculate_lp_features(df)
    lp_score, lp_breakdown = calculate_lp_score(lp_features)
    swap_features = calculate_swap_features(df)
    swap_score, swap_breakdown = calculate_swap_score(swap_features)
    final_score, complete_features = calculate_final_score(lp_score, swap_score, lp_features, swap_features)
    complete_features["score_breakdown"] = {"lp_breakdown": lp_breakdown, "swap_breakdown": swap_breakdown}
    return final_score, complete_features
'''

with open(f"{BASE}/app/main.py", "w") as f:
    f.write(main_py.strip())

with open(f"{BASE}/app/models/dex_model.py", "w") as f:
    f.write(dex_model_py.strip())

print("Wrote:", f"{BASE}/app/main.py")
print("Wrote:", f"{BASE}/app/models/dex_model.py")

Wrote: /content/ai_scoring_server/app/main.py
Wrote: /content/ai_scoring_server/app/models/dex_model.py


In [5]:
import os
os.environ["UVICORN_NO_UVLOOP"] = "1"
os.environ["UVICORN_NO_HTTPTOOLS"] = "1"

import sys, uvicorn, threading, time, requests

# Add your app path
sys.path.append("/content/ai_scoring_server")
from app.main import app

def run_server():
    uvicorn.run(
        app,
        host="0.0.0.0",
        port=8000,
        log_level="info",
        loop="asyncio",  # force asyncio loop
        http="h11"       # avoid httptools
    )

# Run server in background thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# wait a little for server boot
time.sleep(3)

# Test local access
try:
    r = requests.get("http://127.0.0.1:8000/api/v1/health")
    print("Local FastAPI health:", r.text)
except Exception as e:
    print("Local request failed:", e)


INFO:     Started server process [24825]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:41926 - "GET /api/v1/health HTTP/1.1" 200 OK
Local FastAPI health: {"status":"healthy","uptime_seconds":3.06}


In [8]:
# Install cloudflared
!apt-get install cloudflared -qq

import subprocess, re

# Start cloudflared tunnel to port 8000
proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://127.0.0.1:8000"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)

public_url = None

# Read first 50 lines max to avoid blocking forever
for _ in range(50):
    line = proc.stdout.readline()
    if not line:
        break
    decoded = line.decode("utf-8").strip()
    if "trycloudflare.com" in decoded:
        match = re.search(r"https://[0-9a-z\-]+\.trycloudflare\.com", decoded)
        if match:
            public_url = match.group(0)
            print("✅ Public URL:", public_url)
            break


✅ Public URL: https://desired-librarian-cleaning-sucking.trycloudflare.com


In [9]:
import requests, json

base = "http://127.0.0.1:8000"

print("GET /:", requests.get(base + "/").json())
print("GET /api/v1/health:", requests.get(base + "/api/v1/health").json())
print("GET /api/v1/stats:", requests.get(base + "/api/v1/stats").json())


INFO:     127.0.0.1:45102 - "GET / HTTP/1.1" 200 OK
GET /: {'service': 'ai-scoring-server', 'version': '1.0.0', 'status': 'ok'}
INFO:     127.0.0.1:45104 - "GET /api/v1/health HTTP/1.1" 200 OK
GET /api/v1/health: {'status': 'healthy', 'uptime_seconds': 3563.32}
INFO:     127.0.0.1:45118 - "GET /api/v1/stats HTTP/1.1" 200 OK
GET /api/v1/stats: {'processed': 0, 'success': 0, 'failure': 0, 'last_ms': 0.0, 'uptime_seconds': 3563.33}


In [10]:
sample_wallet = {
  "wallet_address": "0x742d35Cc6634C0532925a3b8D4C9db96590e4265",
  "data": [
    {
      "protocolType": "dexes",
      "transactions": [
        {
          "document_id": "507f1f77bcf86cd799439011",
          "action": "swap",
          "timestamp": 1703980800,
          "caller": "0x742d35Cc6634C0532925a3b8D4C9db96590e4265",
          "protocol": "uniswap_v3",
          "poolId": "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
          "poolName": "Uniswap V3 USDC/WETH 0.05%",
          "tokenIn": {"amount": 1000000000, "amountUSD": 1000.0, "address": "0xa0...", "symbol": "USDC"},
          "tokenOut": {"amount": 500000000000000000, "amountUSD": 1000.0, "address": "0xc02...", "symbol": "WETH"}
        },
        {
          "document_id": "507f1f77bcf86cd799439012",
          "action": "deposit",
          "timestamp": 1703980900,
          "caller": "0x742d35Cc6634C0532925a3b8D4C9db96590e4265",
          "protocol": "uniswap_v3",
          "poolId": "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
          "poolName": "Uniswap V3 USDC/WETH 0.05%",
          "token0": {"amount": 500000000, "amountUSD": 500.0, "address": "0xa0...", "symbol": "USDC"},
          "token1": {"amount": 250000000000000000, "amountUSD": 500.0, "address": "0xc02...", "symbol": "WETH"}
        },
        {
          "document_id": "507f1f77bcf86cd799439013",
          "action": "withdraw",
          "timestamp": 1703981900,
          "caller": "0x742d35Cc6634C0532925a3b8D4C9db96590e4265",
          "protocol": "uniswap_v3",
          "poolId": "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640",
          "poolName": "Uniswap V3 USDC/WETH 0.05%",
          "token0": {"amount": 500000000, "amountUSD": 500.0, "address": "0xa0...", "symbol": "USDC"},
          "token1": {"amount": 250000000000000000, "amountUSD": 0.0,   "address": "0xc02...", "symbol": "WETH"}
        }
      ]
    }
  ]
}

resp = requests.post(base + "/api/v1/score", json=sample_wallet)
out = resp.json()
out


INFO:     127.0.0.1:35812 - "POST /api/v1/score HTTP/1.1" 200 OK


{'wallet_address': '0x742d35Cc6634C0532925a3b8D4C9db96590e4265',
 'zscore': '59.034722222222221433',
 'timestamp': 1755672492,
 'processing_time_ms': 20,
 'categories': [{'category': 'dexes',
   'score': 59.03472222222222,
   'transaction_count': 3,
   'features': {'total_deposit_usd': 500.0,
    'total_withdraw_usd': 500.0,
    'num_deposits': 1,
    'num_withdraws': 1,
    'withdraw_ratio': 1.0,
    'avg_hold_time_days': 0.011574074074074073,
    'account_age_days': 0.01273148148148148,
    'unique_pools': 1,
    'total_swap_volume': 1000.0,
    'num_swaps': 1,
    'unique_pools_swapped': 1,
    'avg_swap_size': 1000.0,
    'token_diversity_score': 25,
    'swap_frequency_score': 0.0,
    'lp_score': 55.057870370370374,
    'swap_score': 65.0,
    'final_score': 59.03472222222222}}]}

In [11]:
feat = out["categories"][0]["features"]
checks = {
    "total_deposit_usd": 500.0,
    "total_withdraw_usd": 500.0,
    "num_deposits": 1,
    "num_withdraws": 1,
    "total_swap_volume": 1000.0,
    "num_swaps": 1,
    "unique_pools": 1
}
for k, v in checks.items():
    assert round(float(feat.get(k, 0.0)), 6) == v, (k, feat.get(k), v)
"✅ Feature checks passed"


'✅ Feature checks passed'

In [12]:
import copy, time
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

def make_wallet(i):
    w = copy.deepcopy(sample_wallet)
    w["wallet_address"] = f"0x{i:040x}"
    # jitter timestamps a bit
    base_ts = 1703980800 + i
    w["data"][0]["transactions"][0]["timestamp"] = base_ts
    w["data"][0]["transactions"][1]["timestamp"] = base_ts + 60
    w["data"][0]["transactions"][2]["timestamp"] = base_ts + 120
    # randomize swap volume slightly
    amt = 1000.0 + random.uniform(-100, 100)
    w["data"][0]["transactions"][0]["tokenIn"]["amountUSD"] = amt
    w["data"][0]["transactions"][0]["tokenOut"]["amountUSD"] = amt
    return w

N = 200
payloads = [make_wallet(i) for i in range(N)]

t0 = time.time()
ok = 0
with ThreadPoolExecutor(max_workers=32) as ex:
    futs = [ex.submit(requests.post, base + "/api/v1/score", json=p) for p in payloads]
    for fut in as_completed(futs):
        r = fut.result()
        ok += int(r.status_code == 200)
elapsed = time.time() - t0
rate = N / elapsed * 60.0
{"processed": ok, "seconds": round(elapsed,2), "approx_wallets_per_min": int(rate)}


INFO:     127.0.0.1:57750 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57756 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57768 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57770 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57790 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57792 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57780 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57814 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57804 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57820 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57854 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57826 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57858 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57840 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57874 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:57934

{'processed': 200, 'seconds': 2.35, 'approx_wallets_per_min': 5111}

In [13]:
kafka_consumer_py = r"""
from confluent_kafka import Consumer, Producer
import json, time
from app.models.dex_model import process_wallet_complete, preprocess_dex_transactions

def run_kafka_loop():
    consumer = Consumer({
        "bootstrap.servers": "localhost:9092",
        "group.id": "ai-scoring-service",
        "auto.offset.reset": "earliest",
    })
    producer = Producer({"bootstrap.servers": "localhost:9092"})
    consumer.subscribe(["wallet-transactions"])
    success_topic = "wallet-scores-success"
    failure_topic = "wallet-scores-failure"
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print("Consumer error:", msg.error())
                continue
            payload = json.loads(msg.value())
            try:
                final_score, features = process_wallet_complete(payload)
                if isinstance(features, dict) and features.get("error"):
                    out = {
                        "wallet_address": payload.get("wallet_address",""),
                        "error": features["error"],
                        "timestamp": int(time.time()),
                        "processing_time_ms": 0,
                        "categories": [{"category":"dexes","error":features["error"],"transaction_count":0}]
                    }
                    producer.produce(failure_topic, json.dumps(out).encode())
                else:
                    df = preprocess_dex_transactions(payload)
                    out = {
                        "wallet_address": payload.get("wallet_address",""),
                        "zscore": f"{final_score:.18f}",
                        "timestamp": int(time.time()),
                        "processing_time_ms": 0,
                        "categories": [{
                            "category":"dexes",
                            "score": final_score,
                            "transaction_count": int(len(df)),
                            "features": {k:v for k,v in features.items() if k not in ["score_breakdown","user_tags"]}
                        }]
                    }
                    producer.produce(success_topic, json.dumps(out).encode())
                producer.poll(0)
            except Exception as e:
                out = {"wallet_address": payload.get("wallet_address",""),
                       "error": f"Unhandled error: {e}",
                       "timestamp": int(time.time()),
                       "processing_time_ms": 0,
                       "categories": [{"category":"dexes","error":"Unhandled error","transaction_count":0}]}
                producer.produce(failure_topic, json.dumps(out).encode())
                producer.poll(0)
    finally:
        consumer.close()
"""
print(kafka_consumer_py[:650], "...\n\n# Save this to app/services/kafka_service.py locally to run with a broker.")



from confluent_kafka import Consumer, Producer
import json, time
from app.models.dex_model import process_wallet_complete, preprocess_dex_transactions

def run_kafka_loop():
    consumer = Consumer({
        "bootstrap.servers": "localhost:9092",
        "group.id": "ai-scoring-service",
        "auto.offset.reset": "earliest",
    })
    producer = Producer({"bootstrap.servers": "localhost:9092"})
    consumer.subscribe(["wallet-transactions"])
    success_topic = "wallet-scores-success"
    failure_topic = "wallet-scores-failure"
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None: 
                 ...

# Save this to app/services/kafka_service.py locally to run with a broker.


In [14]:
import re, math

def check_success_schema(resp):
    assert "wallet_address" in resp and isinstance(resp["wallet_address"], str)
    assert "zscore" in resp and re.fullmatch(r"-?\d+\.\d{18}", resp["zscore"])  # 18 decimals
    assert "timestamp" in resp and isinstance(resp["timestamp"], int)
    assert "categories" in resp and isinstance(resp["categories"], list) and len(resp["categories"]) >= 1
    c0 = resp["categories"][0]
    for key in ["category", "score", "transaction_count", "features"]:
        assert key in c0
    assert c0["category"] == "dexes"
    assert isinstance(c0["score"], (int, float))
    assert isinstance(c0["transaction_count"], int)
    assert isinstance(c0["features"], dict)

resp = requests.post(base + "/api/v1/score", json=sample_wallet).json()
check_success_schema(resp)
"✅ Success message schema looks good"


INFO:     127.0.0.1:37166 - "POST /api/v1/score HTTP/1.1" 200 OK


'✅ Success message schema looks good'

In [15]:
dockerfile = """\
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt ./
RUN pip install --no-cache-dir -r requirements.txt
COPY app ./app
EXPOSE 8000
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000"]
"""
reqs = """\
fastapi==0.111.0
uvicorn==0.30.1
pydantic==2.8.2
pandas==2.2.2
numpy==1.26.4
httpx==0.27.0
nest_asyncio==1.6.0
"""

open(f"{BASE}/Dockerfile","w").write(dockerfile)
open(f"{BASE}/requirements.txt","w").write(reqs)
print("Wrote Dockerfile and requirements.txt to", BASE)


Wrote Dockerfile and requirements.txt to /content/ai_scoring_server


In [16]:
import requests, pandas as pd, time

# Cloudflare public URL (replace with your own tunnel URL)
BASE_URL = "https://desired-librarian-cleaning-sucking.trycloudflare.com"

# Example wallet list (replace with real 100 wallets)
wallets = [
    "0x742d35Cc6634C0532925a3b8D4C9db96590e4265",
    "0x06b51c6882b27cb05e712185531c1f74996dd",
    # ... add the rest up to 100
]

results = []

for w in wallets:
    # Build minimal payload (in real case fetch data from The Graph/Etherscan)
    sample_payload = {
        "wallet_address": w,
        "data": []  # TODO: fill with real dex transactions
    }
    try:
        resp = requests.post(BASE_URL + "/api/v1/score", json=sample_payload, timeout=15)
        if resp.status_code == 200:
            results.append(resp.json())
        else:
            results.append({"wallet_address": w, "error": f"status {resp.status_code}"})
    except Exception as e:
        results.append({"wallet_address": w, "error": str(e)})
    time.sleep(0.5)  # be nice to server


INFO:     34.66.242.148:0 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     34.66.242.148:0 - "POST /api/v1/score HTTP/1.1" 200 OK


In [18]:
results = []

for w in wallets:
    # use sample_wallet template but update address
    wallet_data = dict(sample_wallet)
    wallet_data["wallet_address"] = w
    try:
        resp = requests.post(BASE_URL + "/api/v1/score", json=wallet_data, timeout=15)
        results.append(resp.json())
    except Exception as e:
        results.append({"wallet_address": w, "error": str(e)})


INFO:     34.66.242.148:0 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     34.66.242.148:0 - "POST /api/v1/score HTTP/1.1" 200 OK


In [23]:
import random, time
from datetime import datetime, timedelta

TOKENS = [
    {"symbol": "USDC", "address": "0xa0...", "is_stable": True},
    {"symbol": "DAI", "address": "0x6b...", "is_stable": True},
    {"symbol": "USDT", "address": "0xd9...", "is_stable": True},
    {"symbol": "WETH", "address": "0xc0...", "is_stable": False},
    {"symbol": "UNI", "address": "0x1f...", "is_stable": False},
    {"symbol": "MKR", "address": "0x9f...", "is_stable": False},
]

def generate_random_wallet(wallet_address: str, n_tx: int = None):
    """Generate a randomized wallet payload with swaps, deposits, and withdraws."""
    if n_tx is None:
        n_tx = random.randint(3, 10)  # random number of transactions

    now = datetime.now()
    start_date = now - timedelta(days=365*2)  # spread over last 2 years

    transactions = []
    for i in range(n_tx):
        action = random.choice(["swap", "deposit", "withdraw"])
        ts = start_date + timedelta(days=random.randint(0, 365*2),
                                    hours=random.randint(0, 23),
                                    minutes=random.randint(0, 59))
        timestamp = int(ts.timestamp())

        token_in, token_out = random.sample(TOKENS, 2)
        amount_usd = round(random.uniform(100, 10000), 2)

        tx = {
            "document_id": f"tx_{wallet_address}_{i}",
            "action": action,
            "timestamp": timestamp,
            "caller": wallet_address,
            "protocol": "uniswap_v3",
            "poolId": "0xpool123",
            "poolName": "Uniswap V3 Random Pool",
        }

        if action == "swap":
            tx["tokenIn"] = {"amount": random.randint(1, 1_000_000), "amountUSD": amount_usd, "address": token_in["address"], "symbol": token_in["symbol"]}
            tx["tokenOut"] = {"amount": random.randint(1, 1_000_000), "amountUSD": amount_usd, "address": token_out["address"], "symbol": token_out["symbol"]}
        else:
            tx["token0"] = {"amount": random.randint(1, 1_000_000), "amountUSD": amount_usd/2, "address": token_in["address"], "symbol": token_in["symbol"]}
            tx["token1"] = {"amount": random.randint(1, 1_000_000), "amountUSD": amount_usd/2, "address": token_out["address"], "symbol": token_out["symbol"]}

        transactions.append(tx)

    return {
        "wallet_address": wallet_address,
        "data": [{
            "protocolType": "dexes",
            "transactions": transactions
        }]
    }

# Example: generate one wallet
sample_random_wallet = generate_random_wallet("0x00000000000000000000000000000000000000A1")
print(sample_random_wallet["data"][0]["transactions"][:2])  # show first 2 tx


[{'document_id': 'tx_0x00000000000000000000000000000000000000A1_0', 'action': 'withdraw', 'timestamp': 1738817535, 'caller': '0x00000000000000000000000000000000000000A1', 'protocol': 'uniswap_v3', 'poolId': '0xpool123', 'poolName': 'Uniswap V3 Random Pool', 'token0': {'amount': 846273, 'amountUSD': 1246.48, 'address': '0xd9...', 'symbol': 'USDT'}, 'token1': {'amount': 562060, 'amountUSD': 1246.48, 'address': '0xc0...', 'symbol': 'WETH'}}, {'document_id': 'tx_0x00000000000000000000000000000000000000A1_1', 'action': 'swap', 'timestamp': 1702065615, 'caller': '0x00000000000000000000000000000000000000A1', 'protocol': 'uniswap_v3', 'poolId': '0xpool123', 'poolName': 'Uniswap V3 Random Pool', 'tokenIn': {'amount': 440518, 'amountUSD': 4341.58, 'address': '0x6b...', 'symbol': 'DAI'}, 'tokenOut': {'amount': 334782, 'amountUSD': 4341.58, 'address': '0xa0...', 'symbol': 'USDC'}}]


In [24]:
import requests
import pandas as pd

base_url = "http://127.0.0.1:8000"  # or replace with your cloudflared public URL

results = []
N_WALLETS = 100

for i in range(1, N_WALLETS + 1):
    wallet_address = f"0x{i:040x}"  # generates 0x000...001, 0x000...002, etc.
    wallet_payload = generate_random_wallet(wallet_address)

    try:
        resp = requests.post(base_url + "/api/v1/score", json=wallet_payload, timeout=10)
        if resp.status_code == 200:
            results.append(resp.json())
        else:
            results.append({"wallet_address": wallet_address, "error": f"HTTP {resp.status_code}"})
    except Exception as e:
        results.append({"wallet_address": wallet_address, "error": str(e)})

print(f"✅ Collected scores for {len(results)} wallets")

# -----------------------------
# Flatten into DataFrame
# -----------------------------
rows = []
for r in results:
    if "categories" in r:
        cat = r["categories"][0]
        row = {
            "wallet_address": r["wallet_address"],
            "final_score": r.get("zscore", ""),
            "timestamp": r.get("timestamp", ""),
            "transaction_count": cat.get("transaction_count", 0),
        }
        row.update(cat.get("features", {}))  # merge feature breakdown
        rows.append(row)
    else:
        rows.append({"wallet_address": r.get("wallet_address"), "error": r.get("error", "unknown")})

df = pd.DataFrame(rows)
df.to_csv("wallet_scores.csv", index=False)

print("📂 Saved wallet_scores.csv with", len(df), "wallets")
df.head()


INFO:     127.0.0.1:46194 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46200 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46208 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46222 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46236 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46240 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46250 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46252 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46268 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46276 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46284 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46290 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46304 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46308 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46318 - "POST /api/v1/score HTTP/1.1" 200 OK
INFO:     127.0.0.1:46330

,wallet_address,final_score,timestamp,transaction_count,total_deposit_usd,total_withdraw_usd,num_deposits,num_withdraws,withdraw_ratio,avg_hold_time_days,account_age_days,unique_pools,total_swap_volume,num_swaps,unique_pools_swapped,avg_swap_size,token_diversity_score,swap_frequency_score,lp_score,swap_score
0,0x0000000000000000000000000000000000000001,255.324400,1755677974,3,0.000,0.00,0,0,0.000000,0.000000,266.922222,1,22662.20,3,1,7554.066667,45,20.0,270.000000,233.31100
1,0x0000000000000000000000000000000000000002,419.652430,1755677974,3,6212.235,0.00,2,0,0.000000,426.837854,596.156250,1,3916.10,1,1,3916.100000,25,0.0,646.367050,79.58050
2,0x0000000000000000000000000000000000000003,308.967457,1755677974,3,5776.665,2734.77,2,1,0.473417,377.604868,278.434028,1,0.00,0,0,0.000000,0,0.0,514.945761,0.00000
3,0x0000000000000000000000000000000000000004,155.175720,1755677974,3,2287.540,15826.45,1,2,6.918546,236.174313,315.279861,1,0.00,0,0,0.000000,0,0.0,258.626200,0.00000
4,0x0000000000000000000000000000000000000005,242.819390,1755677974,10,510.405,10952.35,2,4,21.458156,132.870833,545.245833,1,21816.05,4,1,5454.012500,75,20.0,225.312150,269.08025


In [27]:
# app/services/db.py
from pymongo import MongoClient
import os

MONGO_URL = os.getenv("MONGODB_URL", "mongodb://localhost:27017")
DB_NAME = os.getenv("MONGODB_DATABASE", "ai_scoring")

client = MongoClient(MONGO_URL)
db = client[DB_NAME]

tokens_collection = db[os.getenv("MONGODB_TOKENS_COLLECTION", "tokens")]
thresholds_collection = db[os.getenv("MONGODB_THRESHOLDS_COLLECTION", "protocol-thresholds-percentiles")]

def get_token_metadata(address: str):
    return tokens_collection.find_one({"address": address})

def get_protocol_threshold(protocol: str):
    return thresholds_collection.find_one({"protocol": protocol})


In [33]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    kafka_bootstrap_servers: str = "localhost:9092"
    kafka_input_topic: str = "wallet-transactions"
    kafka_success_topic: str = "wallet-scores-success"
    kafka_failure_topic: str = "wallet-scores-failure"
    kafka_consumer_group: str = "ai-scoring-service"

    mongodb_url: str = "mongodb://localhost:27017"
    mongodb_database: str = "ai_scoring"
    mongodb_tokens_collection: str = "tokens"
    mongodb_thresholds_collection: str = "protocol-thresholds-percentiles"

    host: str = "0.0.0.0"
    port: int = 8000
    log_level: str = "INFO"
    environment: str = "production"

    class Config:
        env_file = ".env"

settings = Settings()
print(settings.model_dump())



{'kafka_bootstrap_servers': 'localhost:9092', 'kafka_input_topic': 'wallet-transactions', 'kafka_success_topic': 'wallet-scores-success', 'kafka_failure_topic': 'wallet-scores-failure', 'kafka_consumer_group': 'ai-scoring-service', 'mongodb_url': 'mongodb://localhost:27017', 'mongodb_database': 'ai_scoring', 'mongodb_tokens_collection': 'tokens', 'mongodb_thresholds_collection': 'protocol-thresholds-percentiles', 'host': '0.0.0.0', 'port': 8000, 'log_level': 'INFO', 'environment': 'production'}


In [45]:
import os

# Create folders
os.makedirs("app/services", exist_ok=True)
os.makedirs("app/models", exist_ok=True)
os.makedirs("app/utils", exist_ok=True)

# Add __init__.py files so Python treats them as packages
for path in [
    "app/__init__.py",
    "app/services/__init__.py",
    "app/models/__init__.py",
    "app/utils/__init__.py",
]:
    open(path, "w").close()

print("✅ Package structure created")



✅ Package structure created


In [46]:
kafka_service_code = """
from confluent_kafka import Consumer, Producer
import json, time
from app.models.dex_model import process_wallet_complete, preprocess_dex_transactions
from app.config import settings

def run_kafka_loop(stats=None):
    consumer = Consumer({
        "bootstrap.servers": settings.kafka_bootstrap_servers,
        "group.id": settings.kafka_consumer_group,
        "auto.offset.reset": "earliest",
    })
    producer = Producer({"bootstrap.servers": settings.kafka_bootstrap_servers})
    consumer.subscribe([settings.kafka_input_topic])

    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print("Consumer error:", msg.error())
                if stats: stats["errors"] += 1
                continue

            payload = json.loads(msg.value())
            start = time.time()

            try:
                final_score, features = process_wallet_complete(payload)
                elapsed = int((time.time() - start) * 1000)

                if isinstance(features, dict) and features.get("error"):
                    out = {
                        "wallet_address": payload.get("wallet_address",""),
                        "error": features["error"],
                        "timestamp": int(time.time()),
                        "processing_time_ms": elapsed,
                        "categories": [{"category":"dexes","error":features["error"],"transaction_count":0}]
                    }
                    producer.produce(settings.kafka_failure_topic, json.dumps(out).encode())
                    if stats: stats["errors"] += 1
                else:
                    df = preprocess_dex_transactions(payload)
                    out = {
                        "wallet_address": payload.get("wallet_address",""),
                        "zscore": f"{final_score:.18f}",
                        "timestamp": int(time.time()),
                        "processing_time_ms": elapsed,
                        "categories": [{
                            "category":"dexes",
                            "score": final_score,
                            "transaction_count": int(len(df)),
                            "features": features
                        }]
                    }
                    producer.produce(settings.kafka_success_topic, json.dumps(out).encode())
                    if stats: stats["processed"] += 1
                producer.poll(0)

            except Exception as e:
                err = {
                    "wallet_address": payload.get("wallet_address",""),
                    "error": f"Unhandled error: {e}",
                    "timestamp": int(time.time()),
                    "processing_time_ms": 0,
                    "categories": [{"category":"dexes","error":"Unhandled error","transaction_count":0}]
                }
                producer.produce(settings.kafka_failure_topic, json.dumps(err).encode())
                producer.poll(0)
                if stats: stats["errors"] += 1
    finally:
        consumer.close()
"""

with open("app/services/kafka_service.py", "w") as f:
    f.write(kafka_service_code)

print("✅ Created app/services/kafka_service.py")


✅ Created app/services/kafka_service.py


In [50]:
kafka_service_code = """
def run_kafka_loop(stats=None):
    print("✅ Mock Kafka loop started (no real Kafka in Colab).")
"""

with open("/content/ai_scoring_server/app/services/kafka_service.py", "w") as f:
    f.write(kafka_service_code)

print("✅ Created kafka_service.py")


✅ Created kafka_service.py


In [51]:
import sys
sys.path.append("/content/ai_scoring_server")

from app.services.kafka_service import run_kafka_loop
run_kafka_loop()


✅ Mock Kafka loop started (no real Kafka in Colab).


In [54]:
kafka_service_code = """
import time, json
from app.models.dex_model import process_wallet_complete, preprocess_dex_transactions

def run_kafka_loop(stats=None, payload=None):
    print("✅ Mock Kafka loop started (no real Kafka in Colab).")
    if payload is None:
        print("ℹ️ No payload provided, nothing to process.")
        return

    start = time.time()
    try:
        final_score, features = process_wallet_complete(payload)
        elapsed = int((time.time() - start) * 1000)

        if isinstance(features, dict) and features.get("error"):
            out = {
                "wallet_address": payload.get("wallet_address",""),
                "error": features["error"],
                "timestamp": int(time.time()),
                "processing_time_ms": elapsed,
                "categories": [{"category":"dexes","error":features["error"],"transaction_count":0}]
            }
            print("❌ FAILURE MESSAGE:", json.dumps(out, indent=2))
        else:
            df = preprocess_dex_transactions(payload)
            out = {
                "wallet_address": payload.get("wallet_address",""),
                "zscore": f"{final_score:.18f}",
                "timestamp": int(time.time()),
                "processing_time_ms": elapsed,
                "categories": [{
                    "category":"dexes",
                    "score": final_score,
                    "transaction_count": int(len(df)),
                    "features": features
                }]
            }
            print("✅ SUCCESS MESSAGE:", json.dumps(out, indent=2))
    except Exception as e:
        print("🚨 Unhandled error:", str(e))
"""

with open("/content/ai_scoring_server/app/services/kafka_service.py", "w") as f:
    f.write(kafka_service_code)

print("✅ kafka_service.py updated to accept payload argument")


✅ kafka_service.py updated to accept payload argument


In [57]:
from app.services.kafka_service import run_kafka_loop

sample_payload = {
    "wallet_address": "0xABC123",
    "data": [{
        "protocolType": "dexes",
        "transactions": [
            {"document_id":"1", "action":"swap", "timestamp":1699999999,
             "protocol":"UniswapV2", "poolId":"pool-1", "poolName":"USDC/ETH",
             "tokenIn":{"symbol":"USDC","amountUSD":5000},
             "tokenOut":{"symbol":"ETH","amountUSD":4800}},
            {"document_id":"2", "action":"deposit", "timestamp":1700001000,
             "protocol":"UniswapV2", "poolId":"pool-1", "poolName":"USDC/ETH",
             "token0":{"symbol":"USDC","amountUSD":10000},
             "token1":{"symbol":"ETH","amountUSD":9500}},
        ]
    }]
}

run_kafka_loop(payload=sample_payload)


✅ Mock Kafka loop started (no real Kafka in Colab).
✅ SUCCESS MESSAGE: {
  "wallet_address": "0xABC123",
  "zscore": "478.000000000000000000",
  "timestamp": 1755682838,
  "processing_time_ms": 8,
  "categories": [
    {
      "category": "dexes",
      "score": 478.0,
      "transaction_count": 2,
      "features": {
        "total_deposit_usd": 10000.0,
        "total_withdraw_usd": 0.0,
        "num_deposits": 1,
        "num_withdraws": 0,
        "withdraw_ratio": 0.0,
        "avg_hold_time_days": 644.4657221737835,
        "account_age_days": 0.011585648148148149,
        "unique_pools": 1,
        "total_swap_volume": 5000.0,
        "num_swaps": 1,
        "unique_pools_swapped": 1,
        "avg_swap_size": 5000.0,
        "token_diversity_score": 25,
        "swap_frequency_score": 0.0,
        "user_tags": [
          "Medium LP",
          "Long-term Holder",
          "Casual Trader"
        ],
        "lp_score": 740.0,
        "swap_score": 85.0,
        "final_score": 4

In [56]:
import importlib
import sys

# Make sure path is there
sys.path.append("/content/ai_scoring_server")

# Reload module to pick up latest code
import app.services.kafka_service as kafka_service
importlib.reload(kafka_service)

# Now the new function signature will be used
kafka_service.run_kafka_loop(payload=sample_payload)


✅ Mock Kafka loop started (no real Kafka in Colab).
✅ SUCCESS MESSAGE: {
  "wallet_address": "0xABC123",
  "zscore": "478.000000000000000000",
  "timestamp": 1755682702,
  "processing_time_ms": 14,
  "categories": [
    {
      "category": "dexes",
      "score": 478.0,
      "transaction_count": 2,
      "features": {
        "total_deposit_usd": 10000.0,
        "total_withdraw_usd": 0.0,
        "num_deposits": 1,
        "num_withdraws": 0,
        "withdraw_ratio": 0.0,
        "avg_hold_time_days": 644.464146558554,
        "account_age_days": 0.011585648148148149,
        "unique_pools": 1,
        "total_swap_volume": 5000.0,
        "num_swaps": 1,
        "unique_pools_swapped": 1,
        "avg_swap_size": 5000.0,
        "token_diversity_score": 25,
        "swap_frequency_score": 0.0,
        "user_tags": [
          "Medium LP",
          "Long-term Holder",
          "Casual Trader"
        ],
        "lp_score": 740.0,
        "swap_score": 85.0,
        "final_score": 4

In [58]:
# app/main.py
from fastapi import FastAPI
from app.services.kafka_service import run_kafka_loop
import threading, time

app = FastAPI()
stats = {"processed": 0, "errors": 0, "start_time": time.time()}

@app.get("/")
def info():
    return {"service": "AI DeFi Scoring", "status": "running"}

@app.get("/api/v1/health")
def health():
    return {"status": "healthy"}

@app.get("/api/v1/stats")
def get_stats():
    uptime = int(time.time() - stats["start_time"])
    return {**stats, "uptime_sec": uptime}


In [60]:
from fastapi import Body
from fastapi.responses import JSONResponse

@app.post("/api/v1/score")
def score_wallet(payload: dict = Body(...)):
    result = run_kafka_loop(payload=payload, return_json=True)
    return JSONResponse(content=result)


In [61]:
TOKENS = {"USDC": {"decimals": 6, "address": "0xA0b86991..."}}
PROTOCOL_THRESHOLDS = {"UniswapV2": {"p95": 700}}

def get_token_metadata(address: str):
    return TOKENS.get(address, {})

def get_protocol_threshold(protocol: str):
    return PROTOCOL_THRESHOLDS.get(protocol, {})


In [62]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    host: str = "0.0.0.0"
    port: int = 8000
    log_level: str = "INFO"
    environment: str = "dev"
settings = Settings()


In [63]:
# app/services/kafka_service.py
from datetime import datetime
import time
from app.models.dex_model import process_wallet_complete, preprocess_dex_transactions

def run_kafka_loop(payload: dict = None, return_json: bool = False):
    """
    Mock Kafka consumer that processes a wallet payload.
    If return_json=True, returns the scoring result dict instead of just printing.
    """
    print("✅ Mock Kafka loop started (no real Kafka in Colab).")

    if payload:
        t0 = time.time()
        try:
            final_score, features = process_wallet_complete(payload)

            if isinstance(features, dict) and features.get("error"):
                failure_msg = {
                    "wallet_address": payload.get("wallet_address", ""),
                    "error": features["error"],
                    "timestamp": int(datetime.now().timestamp()),
                    "processing_time_ms": int((time.time() - t0) * 1000),
                    "categories": [{
                        "category": "dexes",
                        "error": features["error"],
                        "transaction_count": 0
                    }]
                }
                print("❌ FAILURE MESSAGE:", failure_msg)
                return failure_msg if return_json else None

            df = preprocess_dex_transactions(payload)
            success_msg = {
                "wallet_address": payload.get("wallet_address", ""),
                "zscore": f"{final_score:.18f}",
                "timestamp": int(datetime.now().timestamp()),
                "processing_time_ms": int((time.time() - t0) * 1000),
                "categories": [{
                    "category": "dexes",
                    "score": final_score,
                    "transaction_count": int(len(df)),
                    "features": {k: v for k, v in features.items()
                                 if k not in ["score_breakdown", "user_tags"]}
                }]
            }
            print("✅ SUCCESS MESSAGE:", success_msg)
            return success_msg if return_json else None

        except Exception as e:
            failure_msg = {
                "wallet_address": payload.get("wallet_address", ""),
                "error": f"Unhandled error: {e}",
                "timestamp": int(datetime.now().timestamp()),
                "processing_time_ms": int((time.time() - t0) * 1000),
                "categories": [{
                    "category": "dexes",
                    "error": "Unhandled error",
                    "transaction_count": 0
                }]
            }
            print("❌ FAILURE MESSAGE:", failure_msg)
            return failure_msg if return_json else None

    return None


In [64]:
import importlib, app.main as main
importlib.reload(main)

sample_payload = {
    "wallet_address": "0xABC123",
    "data": [
        {
            "protocolType": "dexes",
            "transactions": [
                {"action": "deposit", "timestamp": 1609459200,
                 "token0": {"amountUSD": 10000, "symbol": "USDC"}},
                {"action": "swap", "timestamp": 1609462800,
                 "tokenIn": {"amountUSD": 5000, "symbol": "USDC"},
                 "tokenOut": {"amountUSD": 5000, "symbol": "ETH"}}
            ]
        }
    ]
}

# Direct call
resp = main.score_wallet(payload=sample_payload)
print(resp.body.decode())


{"wallet_address":"0xABC123","zscore":"478.000000000000000000","timestamp":1755683437,"processing_time_ms":7,"categories":[{"category":"dexes","score":478.0,"transaction_count":2,"features":{"total_deposit_usd":10000.0,"total_withdraw_usd":0.0,"num_deposits":1,"num_withdraws":0,"withdraw_ratio":0.0,"avg_hold_time_days":1692.410150707927,"account_age_days":0.041666666666666664,"unique_pools":1,"total_swap_volume":5000.0,"num_swaps":1,"unique_pools_swapped":1,"avg_swap_size":5000.0,"token_diversity_score":25,"swap_frequency_score":0.0,"lp_score":740.0,"swap_score":85.0,"final_score":478.0}}]}


In [65]:
# ✅ FastAPI TestClient simulation inside Colab
from fastapi.testclient import TestClient
import importlib, app.main as main

# Reload your app to ensure latest changes
importlib.reload(main)

# Create a test client
client = TestClient(main.app)

# Sample payload
sample_payload = {
    "wallet_address": "0xABC123",
    "data": [
        {
            "protocolType": "dexes",
            "transactions": [
                {
                    "action": "deposit",
                    "timestamp": 1609459200,
                    "token0": {"amountUSD": 10000, "symbol": "USDC"}
                },
                {
                    "action": "swap",
                    "timestamp": 1609462800,
                    "tokenIn": {"amountUSD": 5000, "symbol": "USDC"},
                    "tokenOut": {"amountUSD": 5000, "symbol": "ETH"}
                }
            ]
        }
    ]
}

# Call the API endpoint just like a real HTTP request
resp = client.post("/api/v1/score", json=sample_payload)

print("Status Code:", resp.status_code)
print("Response JSON:")
print(resp.json())


Status Code: 200
Response JSON:
{'wallet_address': '0xABC123', 'zscore': '478.000000000000000000', 'timestamp': 1755683783, 'processing_time_ms': 7, 'categories': [{'category': 'dexes', 'score': 478.0, 'transaction_count': 2, 'features': {'total_deposit_usd': 10000.0, 'total_withdraw_usd': 0.0, 'num_deposits': 1, 'num_withdraws': 0, 'withdraw_ratio': 0.0, 'avg_hold_time_days': 1692.414162325045, 'account_age_days': 0.041666666666666664, 'unique_pools': 1, 'total_swap_volume': 5000.0, 'num_swaps': 1, 'unique_pools_swapped': 1, 'avg_swap_size': 5000.0, 'token_diversity_score': 25, 'swap_frequency_score': 0.0, 'lp_score': 740.0, 'swap_score': 85.0, 'final_score': 478.0}}]}


In [66]:
# ✅ Test all endpoints in one go
from fastapi.testclient import TestClient
import importlib, app.main as main

# Reload to ensure fresh app
importlib.reload(main)
client = TestClient(main.app)

# --- Test Root ---
print("📍 GET /")
print(client.get("/").json(), "\n")

# --- Test Health ---
print("📍 GET /api/v1/health")
print(client.get("/api/v1/health").json(), "\n")

# --- Test Stats (before scoring) ---
print("📍 GET /api/v1/stats (before scoring)")
print(client.get("/api/v1/stats").json(), "\n")

# --- Test Score ---
sample_payload = {
    "wallet_address": "0xABC123",
    "data": [
        {
            "protocolType": "dexes",
            "transactions": [
                {
                    "action": "deposit",
                    "timestamp": 1609459200,
                    "token0": {"amountUSD": 10000, "symbol": "USDC"}
                },
                {
                    "action": "swap",
                    "timestamp": 1609462800,
                    "tokenIn": {"amountUSD": 5000, "symbol": "USDC"},
                    "tokenOut": {"amountUSD": 5000, "symbol": "ETH"}
                }
            ]
        }
    ]
}
print("📍 POST /api/v1/score")
resp = client.post("/api/v1/score", json=sample_payload)
print(resp.json(), "\n")

# --- Test Stats (after scoring) ---
print("📍 GET /api/v1/stats (after scoring)")
print(client.get("/api/v1/stats").json(), "\n")


📍 GET /
{'service': 'ai-scoring-server', 'version': '1.0.0', 'status': 'ok'} 

📍 GET /api/v1/health
{'status': 'healthy', 'uptime_seconds': 0.01} 

📍 GET /api/v1/stats (before scoring)
{'processed': 0, 'success': 0, 'failure': 0, 'last_ms': 0.0, 'uptime_seconds': 0.01} 

📍 POST /api/v1/score
{'wallet_address': '0xABC123', 'zscore': '478.000000000000000000', 'timestamp': 1755683932, 'processing_time_ms': 7, 'categories': [{'category': 'dexes', 'score': 478.0, 'transaction_count': 2, 'features': {'total_deposit_usd': 10000.0, 'total_withdraw_usd': 0.0, 'num_deposits': 1, 'num_withdraws': 0, 'withdraw_ratio': 0.0, 'avg_hold_time_days': 1692.4158856936103, 'account_age_days': 0.041666666666666664, 'unique_pools': 1, 'total_swap_volume': 5000.0, 'num_swaps': 1, 'unique_pools_swapped': 1, 'avg_swap_size': 5000.0, 'token_diversity_score': 25, 'swap_frequency_score': 0.0, 'lp_score': 740.0, 'swap_score': 85.0, 'final_score': 478.0}}]} 

📍 GET /api/v1/stats (after scoring)
{'processed': 1, 'suc

In [67]:
# tests/test_api.py
import pytest
from fastapi.testclient import TestClient
import app.main as main

client = TestClient(main.app)

# ---------- Fixtures ----------
@pytest.fixture
def sample_payload():
    return {
        "wallet_address": "0xABC123",
        "data": [
            {
                "protocolType": "dexes",
                "transactions": [
                    {
                        "action": "deposit",
                        "timestamp": 1609459200,
                        "token0": {"amountUSD": 10000, "symbol": "USDC"}
                    },
                    {
                        "action": "swap",
                        "timestamp": 1609462800,
                        "tokenIn": {"amountUSD": 5000, "symbol": "USDC"},
                        "tokenOut": {"amountUSD": 5000, "symbol": "ETH"}
                    }
                ]
            }
        ]
    }

# ---------- Tests ----------
def test_root():
    resp = client.get("/")
    assert resp.status_code == 200
    data = resp.json()
    assert data["service"] == "ai-scoring-server"
    assert data["status"] == "ok"

def test_health():
    resp = client.get("/api/v1/health")
    assert resp.status_code == 200
    data = resp.json()
    assert data["status"] == "healthy"
    assert "uptime_seconds" in data

def test_stats():
    resp = client.get("/api/v1/stats")
    assert resp.status_code == 200
    data = resp.json()
    assert "processed" in data
    assert "errors" in data
    assert "uptime_sec" in data or "uptime_seconds" in data

def test_score(sample_payload):
    resp = client.post("/api/v1/score", json=sample_payload)
    assert resp.status_code == 200
    data = resp.json()
    assert data["wallet_address"] == "0xABC123"
    assert "categories" in data
    assert data["categories"][0]["category"] == "dexes"
    assert "features" in data["categories"][0]
    assert "final_score" in data["categories"][0]["features"]
